# IDB_drilling_signal

## Import packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

import gc
import IPython
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io as scio
import soundfile
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow import keras

HOST_path = "/content/drive/MyDrive"

## Load signal data

In [ ]:
def read_matFile(signal_path):
  signal_file = scio.loadmat(signal_path)
  signal_value = signal_file['samples']
  signal_value = np.squeeze(signal_value)
  return signal_value

def read_wavFile(signal_path):
  signal_rate, signal_value = scio.wavfile.read(signal_path)
  return signal_value, signal_rate

def read_soundFile(signal_path):
  signal_value, signal_rate = librosa.load(signal_path, sr=None, mono=True, offset=0.0, duration=None)
  return signal_value, signal_rate

RATE = 8192

S32h = read_matFile(os.path.join(HOST_path, "IDB_drilling_signal/rock_drilling_signal/experiment1/matFile/3-2h.mat"))
S41h = read_matFile(os.path.join(HOST_path, "IDB_drilling_signal/rock_drilling_signal/experiment1/matFile/4-1h.mat"))
S42h = read_matFile(os.path.join(HOST_path, "IDB_drilling_signal/rock_drilling_signal/experiment1/matFile/4-2h.mat"))
S43b = read_matFile(os.path.join(HOST_path, "IDB_drilling_signal/rock_drilling_signal/experiment1/matFile/4-3b.mat"))

print("RATE: ", RATE, "Hz (default in Matlab)")
print("---------------------------------------")
print("S32h : ", S32h.shape, len(S32h)/RATE, 's')
print("S41h : ", S41h.shape, len(S41h)/RATE, 's')
print("S42h : ", S42h.shape, len(S42h)/RATE, 's')
print("S43b : ", S43b.shape, len(S43b)/RATE, 's')

## Convert signal data

In [ ]:
dir_path = os.path.join(HOST_path, "IDB_drilling_signal/rock_drilling_signal/experiment1/matFile")
path_list = os.listdir(dir_path)
path_list.sort()
for i,j in enumerate(path_list):
    name = os.path.splitext(j)
    mat_path = os.path.join(dir_path, j)
    mat_file = read_matFile(mat_path)
    wav_path = os.path.join(dir_path, name[0]+".wav")
    soundfile.write(wav_path, mat_file, RATE)

## Plot signal data

In [ ]:
def displayWaveform(signal, rate):
  plt.figure(figsize=(20,6))
  max = np.max(np.absolute(signal))*1.2
  time = np.arange(0, len(SIGNAL)) / RATE
  plt.plot(time, signal)
  plt.title("Time domain waveform of speech signal")
  plt.xlabel("time (s)")
  plt.ylabel("amplitude")
  plt.xlim(0,len(SIGNAL)/RATE)
  ymin, ymax = plt.ylim()
  ylim = np.maximum(np.abs(ymin), np.abs(ymax))
  plt.ylim(-ylim, ylim)

def displaySpectrum(signal, rate):
  plt.figure(figsize=(20,6)) 
  s = np.fft.fft(signal)
  m = np.abs(s)
  n = len(signal)
  f = np.fft.fftfreq(n, 1/rate)
  plt.plot(f[:n//2],m[:n//2])
  plt.title("Frequency domain spectral line of speech signal")
  plt.xlabel("Frequency (Hz)")
  plt.ylabel("amplitude")
  plt.xlim(0, rate//2)

def displaySpectrogram(signal, rate, fftlen):    
  plt.figure(figsize=(8,6))
  plt.specgram(signal, NFFT=fftlen, Fs=rate, noverlap=int(fftlen*0.25), window=np.hanning(fftlen))
  plt.title('Linear-frequency power spectrogram')
  plt.xlabel('time (s)')
  plt.ylabel('Frequency (Hz)')
  plt.colorbar(format="%+2.0f dB")

def plot_wave(signal, rate):  
  displayWaveform(signal, rate)
  displaySpectrum(signal, rate)
  displaySpectrogram(signal, rate, fftlen=512)

SIGNAL = S32h[-40*RATE:]
plot_wave(SIGNAL, RATE)

import IPython

IPython.display.Audio(data=SIGNAL, rate=RATE)

## Classification

### Data pre-treatment

| signal| mat2wav | pick |
| :---: | :---: | :---: |
| S32h | 00:00-17:36 | 02:00-12:00 |
| S41h | 01:21-14:03 | 02:00-12:00 |
| S42h | 00:00-21:24 | 02:00-12:00 |
| S43b | 01:01-37:31 | 02:00-12:00 14:00-24:00 26:00-36:00 |

In [ ]:
def cut_signal(signal_series):
  sample_size = int(RATE/2)         # 4096
  sample_step = int(sample_size/4)  # 512
  sample_data = []
  for i in range((len(signal_series)-sample_size) // sample_step):
    sample_data.append(signal_series[i*sample_step : (i*sample_step+sample_size)])
  sample_data = np.stack(sample_data)
  sample_data = np.squeeze(sample_data)
  # np.random.seed(42)
  # np.random.shuffle(sample_data)  
  return sample_data

S32hCX = cut_signal(S32h[120*RATE:720*RATE])
S41hCX = cut_signal(S41h[120*RATE:720*RATE])
S42hCX = cut_signal(S42h[120*RATE:720*RATE])
S43bCX1 = cut_signal(S43b[120*RATE:720*RATE])
S43bCX2 = cut_signal(S43b[840*RATE:1440*RATE])
S43bCX3 = cut_signal(S43b[1560*RATE:2160*RATE])

S32hCY = np.repeat(0, len(S32hCX))
S41hCY = np.repeat(0, len(S41hCX))
S42hCY = np.repeat(0, len(S42hCX))
S43bCY1 = np.repeat(1, len(S43bCX1))
S43bCY2 = np.repeat(1, len(S43bCX2))
S43bCY3 = np.repeat(1, len(S43bCX3))

X_set = np.concatenate((S32hCX, S41hCX, S42hCX, S43bCX1, S43bCX2, S43bCX3), axis=0)
X_set = X_set.reshape((X_set.shape[0], X_set.shape[1], 1))

Y_set = np.concatenate((S32hCY, S41hCY, S42hCY, S43bCY1, S43bCY2, S43bCY3), axis=0)
num_classes = len(np.unique(Y_set))
Y_set = to_categorical(Y_set, num_classes=num_classes)

x_train, x_test, y_train, y_test = train_test_split(X_set, Y_set, test_size=0.33, random_state=42)

del S32hCX, S41hCX, S42hCX, S43bCX1, S43bCX2, S43bCX3, X_set
del S32hCY, S41hCY, S42hCY, S43bCY1, S43bCY2, S43bCY3, Y_set
gc.collect()

### Model establishment

In [ ]:
def make_model():
    model = Sequential()
    model.add(Conv1D(64, 3, input_shape=x_train.shape[1:], padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(64, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(128, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(128, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(256, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(256, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(256, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(4096))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='sigmoid'))
    return model

model = make_model()
model.summary()
keras.utils.plot_model(model, show_shapes=True)

### Complete & Train

In [ ]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy',
                       keras.metrics.AUC(),
                       keras.metrics.Precision(),
                       keras.metrics.Recall()])

callbacks = [
             keras.callbacks.ModelCheckpoint("best_model.hdf5",
                                             monitor="loss",
                                             mode="min",
                                             save_best_only=True),
             keras.callbacks.EarlyStopping(monitor="val_loss",
                                           mode="min",
                                           verbose=1,
                                           patience=10,
                                           restore_best_weights=True), 
             keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                               factor=0.2,
                                               patience=2,
                                               min_lr=0.000001),
            # keras.callbacks.TensorBoard(log_dir=os.path.join(HOST_path, "IDB_drilling_signal/outputs/C_logs"),
            #                             histogram_freq=1,
            #                             write_graph=True, 
            #                             write_images=True),
            ]

history = model.fit(x_train, y_train,
                    validation_split=0.2, 
                    epochs=200, 
                    batch_size=64, 
                    callbacks=callbacks, 
                    shuffle=True)

### Show model results

In [ ]:
y_test_true = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(model.predict(x_test), axis=1)
test_loss, test_accuracy, test_auc, test_precision, test_recall = model.evaluate(x_test, y_test)

cm = confusion_matrix(y_test_true, y_test_pred)
tn, fp, fn, tp = cm.ravel()
precision = tp/(tp + fp)
recall = tp/(tp + tn)

### Save model results

In [ ]:
eval_dict = {"y_test_true":y_test_true, "y_test_pred":y_test_pred, "test_loss":test_loss, "test_accuracy":test_accuracy, "test_auc":test_auc, "test_precision":test_precision, "test_recall":test_recall, "cm":cm, "cm_tn":tn, "cm_fp":fp, "cm_fn":fn, "cm_tp":tp, "cm_precision":precision, "cm_recall":recall}

model.save(os.path.join(HOST_path, "IDB_drilling_signal/outputs/C_model.hdf5"))
np.save(os.path.join(HOST_path, "IDB_drilling_signal/outputs/C_history.npy"), history.history)
np.save(os.path.join(HOST_path, "IDB_drilling_signal/outputs/C_evaluation.npy"), eval_dict)

### Load model results

In [ ]:
# from keras.models import load_model

# model = load_model(os.path.join(HOST_path, "IDB_drilling_signal/outputs/C_model.hdf5"))
# hist = np.load(os.path.join(HOST_path, "IDB_drilling_signal/outputs/C_history.npy"), allow_pickle=True).item()
# eval_dict = np.load(os.path.join(HOST_path, "IDB_drilling_signal/outputs/C_evaluation.npy"), allow_pickle=True)

### Plot model results

In [ ]:
def plot_history_metrics(history):
  total_plots = len(history)
  cols = total_plots // 2
  rows = total_plots // cols
  if total_plots % cols != 0:
    rows += 1
  pos = range(1, total_plots + 1)
  plt.figure(figsize=(15, 10))
  for i, (key, value) in enumerate(history.items()):
    plt.subplot(rows, cols, pos[i])
    plt.plot(range(len(value)), value)
    plt.title(str(key))
  plt.savefig(os.path.join(HOST_path, "IDB_drilling_signal/outputs/C_evaluation.png"))
 
plot_history_metrics(history.history)

## Regression

### Data pre-treatment

| signal| table | mat2wav |
| :---: | :---: | :---: |
| S11 | 10:59:18-11:01:50 | 00:00-02:32 |
| S12 | 09:16:44-09:18:23 | 00:00-01:39 |
| S13 | 09:21:37-09:23:05 | 00:00-01:28 |
| S14 | 09:45:41-09:46:24 | 00:00-00:43 |
| S15 | 09:49:43-09:50:15 | 00:00-00:32 |
| S16 | 10:02:30-10:03:30 | 00:00-01:00 |
| S17 | 10:27:57-10:29:26 | 00:00-01:29 |
| S18 | 10:35:29-10:36:47 | 00:00-01:18 |
| S19 | 10:40:35-10:41:28 | 00:00-00:53 |

In [ ]:
S11RX = S11[0:152*RATE]
S12RX = S11[0:99*RATE]
S13RX = S11[0:88*RATE]
S14RX = S11[0:43*RATE]
S15RX = S11[0:32*RATE]
S16RX = S11[0:60*RATE]
S17RX = S11[0:89*RATE]
S18RX = S11[0:78*RATE]
S19RX = S11[0:53*RATE]

def load_table(table_path, time_start, time_end):
  SIGNAL_pd = pd.read_excel(table_path)
  display(SIGNAL_pd)
  SIGNAL_arr = SIGNAL_pd.values[time_start : time_end,1:-1]
  print(type(SIGNAL_arr), SIGNAL_arr.shape, len(SIGNAL_arr)-1, "s")
  print(SIGNAL_arr)
  return SIGNAL_arr

S11RY = load_table(os.path.join(HOST_path, "IDB_drilling_signal/Rock_drilling_signal/experiment2/1_1.xls"), 81, 82+152)
S12RY = load_table(os.path.join(HOST_path, "IDB_drilling_signal/Rock_drilling_signal/experiment2/1_2.xls"), 47, 51+96)
S13RY = load_table(os.path.join(HOST_path, "IDB_drilling_signal/Rock_drilling_signal/experiment2/1_3.xls"), 40, 41+88)
S14RY = load_table(os.path.join(HOST_path, "IDB_drilling_signal/Rock_drilling_signal/experiment2/1_4.xls"), 44, 45+43)
S15RY = load_table(os.path.join(HOST_path, "IDB_drilling_signal/Rock_drilling_signal/experiment2/1_5.xls"), 46, 47+32)
S16RY = load_table(os.path.join(HOST_path, "IDB_drilling_signal/Rock_drilling_signal/experiment2/1_6.xls"), 573, 574+60)
S17RY = load_table(os.path.join(HOST_path, "IDB_drilling_signal/Rock_drilling_signal/experiment2/1_7.xls"), 60, 61+89)
S18RY = load_table(os.path.join(HOST_path, "IDB_drilling_signal/Rock_drilling_signal/experiment2/1_8.xls"), 32, 33+78)
S19RY = load_table(os.path.join(HOST_path, "IDB_drilling_signal/Rock_drilling_signal/experiment2/1_9.xls"), 38, 39+53)

def enlarge_value(array, length):
  L = array.shape[0]
  W = array.shape[1]
  new_section = np.zeros([(L-1)*length, W-1])
  for i in range(L - 1):   
    new_second = np.zeros([length, W-1])
    for j in range(W - 1):
      start_value = float(array[i, j+1])
      end_value = float(array[i+1, j+1])
      new_second[:,j] = np.linspace(start_value, end_value, length + 2)[1:-1]  
    new_section[i*length:(i+1)*length,:] = new_second
  return new_section

S11RY = enlarge_value(S11RY, RATE)
S12RY = enlarge_value(S12RY, RATE)
S13RY = enlarge_value(S13RY, RATE)
S14RY = enlarge_value(S14RY, RATE)
S15RY = enlarge_value(S15RY, RATE)
S16RY = enlarge_value(S16RY, RATE)
S17RY = enlarge_value(S17RY, RATE)
S18RY = enlarge_value(S18RY, RATE)
S19RY = enlarge_value(S19RY, RATE)

def pick_value(signal, label):
  sample_size = int(RATE/2)         # 4096
  sample_step = int(sample_size/4)  # 512
  sample_data = []
  sample_idx = []
  for i in range((len(signal)-sample_size) // sample_step):
    sample_data.append(signal[i*sample_step : (i*sample_step+sample_size)])
    sample_idx.append(i*sample_step + sample_size//2)
  sample_data = np.stack(sample_data)
  sample_data = np.squeeze(sample_data)
  sample_label = label[sample_idx]
  return sample_data, sample_label

S11RX, S11RY = pick_value(S11RX, S11RY)
S12RX, S12RY = pick_value(S12RX, S12RY)
S13RX, S13RY = pick_value(S13RX, S13RY)
S14RX, S14RY = pick_value(S14RX, S14RY)
S15RX, S15RY = pick_value(S15RX, S15RY)
S16RX, S16RY = pick_value(S16RX, S16RY)
S17RX, S17RY = pick_value(S17RX, S17RY)
S18RX, S18RY = pick_value(S18RX, S18RY)
S19RX, S19RY = pick_value(S19RX, S19RY)

X_set = np.concatenate((S11RX, S12RX, S13RX, S14RX, S15RX, S16RX, S17RX, S18RX, S19RX), axis=0)
X_set = X_set.reshape((X_set.shape[0], X_set.shape[1], 1))

Y_set = np.concatenate((S11RY, S12RY, S13RY, S14RY, S15RY, S16RY, S17RY, S18RY, S19RY), axis=0)
num_targets = Y_set.shape[1]

x_train, x_test, y_train, y_test = train_test_split(X_set, Y_set, test_size=0.33, random_state=42)

del S32hRX, S41hRX, S42hRX, S43bRX, S32hRY, S41hRY, S42hRY, S43bRY, X_set, Y_set
gc.collect()

### Model establishment

In [ ]:
def make_model():
    model = Sequential()
    model.add(Conv1D(64, 3, input_shape=x_train.shape[1:], padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(64, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(128, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(128, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(256, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(256, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(256, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D(512, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(4096))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_targets, activation='linear'))
    return model

model = make_model()
model.summary()
keras.utils.plot_model(model, show_shapes=True)

### Complete & Train

In [ ]:
model.compile(optimizer='adam', loss='mse')

callbacks = [
             keras.callbacks.ModelCheckpoint("best_model.hdf5",
                                             monitor="loss",
                                             mode="min",
                                             save_best_only=True),
             keras.callbacks.EarlyStopping(monitor="val_loss",
                                           mode="min",
                                           verbose=1,
                                           patience=10,
                                           restore_best_weights=True), 
             keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                               factor=0.2,
                                               patience=2,
                                               min_lr=0.000001),
            # keras.callbacks.TensorBoard(log_dir=os.path.join(HOST_path, "IDB_drilling_signal/outputs/C_logs"),
            #                             histogram_freq=1,
            #                             write_graph=True, 
            #                             write_images=True),
            ]

history = model.fit(x_train, y_train,
                    validation_split=0.2, 
                    epochs=200, 
                    batch_size=64, 
                    callbacks=callbacks, 
                    shuffle=True)

### Show model results

In [ ]:
y_test_true = y_test
y_test_pred = model.predict(x_test)
test_loss = model.evaluate(x_test, y_test)

### Save model results

In [ ]:
eval_dict = {"y_test_true":y_test_true, "y_test_pred":y_test_pred, "test_loss":test_loss}

model.save(os.path.join(HOST_path, "IDB_drilling_signal/outputs/R_model.hdf5"))
np.save(os.path.join(HOST_path, "IDB_drilling_signal/outputs/R_history.npy"), history.history)
np.save(os.path.join(HOST_path, "IDB_drilling_signal/outputs/R_evaluation.npy"), eval_dict)

### Load model results

In [ ]:
# from keras.models import load_model

# model = load_model("/home/svu/e1097232/IDB_drilling_signal/outputs/R_model.hdf5")
# hist = np.load("/home/svu/e1097232/IDB_drilling_signal/outputs/R_history.npy", allow_pickle=True)
# eval_dict = np.load("/home/svu/e1097232/IDB_drilling_signal/outputs/R_evaluation.npy", allow_pickle=True)

### Plot model results

In [ ]:
# Plot Loss-Epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train set', 'Test set'], loc='upper left')
plt.savefig(os.path.join(HOST_path, "IDB_drilling_signal/outputs/R_loss.png"))
plt.show()